In [1]:
import pandas as pd
import requests
import sqlalchemy
from sqlalchemy import create_engine
import sqlite3

In [2]:
url="https://api.census.gov/data/2019/pep/charagegroups?"


In [3]:
all_population="000"
age_65_up="026"
query_url=url+ "get=NAME,POP&for=county:*&in=state:12&AGEGROUP="
all_pop_query=query_url+all_population
age_65_up_query=query_url+age_65_up
results_cols=["County, State","Population","Age Group","State Code","County Code"]

In [4]:
all_pop_results=requests.get(all_pop_query).json()
age_65_up_results=requests.get(age_65_up_query).json()

In [5]:
age_65_up_df=pd.DataFrame(age_65_up_results,columns=results_cols)
age_65_up_df.columns=age_65_up_df.iloc[0]
age_65_up_df=age_65_up_df.tail(-1)
age_65_up_df

,NAME,POP,AGEGROUP,state,county
1,"Washington County, Florida",4380,26,12,133
2,"Duval County, Florida",138594,26,12,031
3,"Bradford County, Florida",5064,26,12,007
4,"Brevard County, Florida",145752,26,12,009
5,"Clay County, Florida",35844,26,12,019
...,...,...,...,...,...
63,"Hernando County, Florida",53348,26,12,053
64,"Hillsborough County, Florida",214035,26,12,057
65,"Okaloosa County, Florida",34376,26,12,091
66,"Taylor County, Florida",4525,26,12,123


In [6]:
all_pop_df=pd.DataFrame(all_pop_results,columns=results_cols)
all_pop_df.columns=all_pop_df.iloc[0]
all_pop_df=all_pop_df.tail(-1)

In [7]:
pop_percent_df=pd.merge(all_pop_df,age_65_up_df,on='county',how='outer')

In [8]:
pop_percent_df[['County','State']]=pop_percent_df['NAME_x'].str.split(', ',expand=True)
pop_percent_df.drop(columns=['NAME_x','NAME_y','AGEGROUP_x','AGEGROUP_y','state_x','state_y','State'],inplace=True)

In [9]:
pop_percent_df=pop_percent_df.rename(columns={'county':'County_ID','POP_x':'Total_Population','POP_y':'Elderly_Population','state':'State_ID'})
data_types={'Total_Population':'int','County_ID':'int','Elderly_Population':'int'}
pop_percent_df=pop_percent_df.astype(data_types)
pop_percent_df['Percent_Elderly']=(pop_percent_df['Elderly_Population']/pop_percent_df['Total_Population'])*100
pop_percent_df['Percent_Elderly'] = pop_percent_df['Percent_Elderly'].map("{:.2f}".format)
pop_percent_df['County'] = pop_percent_df['County'].str.replace(' County', '')

In [10]:
county_table=pop_percent_df[['County_ID','County']]

In [11]:
city_county_df=pd.read_csv('Florida City County and Coordinates.csv')
city_county_df=city_county_df.rename(columns={"CITY":"City","COUNTY":"County","LATITUDE":"Latitude","LONGITUDE":"Longitude"})
fl_city_df=pd.merge(pop_percent_df,city_county_df,on='County',how='left')
db_file = 'elderly.sqlite'
conn = sqlite3.connect(db_file)
pop_percent_df.to_sql('data', conn, if_exists='replace', index=False)

67